<a href="https://colab.research.google.com/github/daveselinger/covid-19-hackathon/blob/master/by_state_covid_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!git clone https://github.com/CSSEGISandData/COVID-19
!head -5 COVID-19/who_covid_19_situation_reports/who_covid_19_sit_rep_time_series/who_covid_19_sit_rep_time_series.csv

In [2]:
!pip install -U plotly
!pip install plotly-geo
!pip install geopandas==0.3.0
!pip install pyshp==1.2.10

     |████████████████████████████████| 7.1MB 4.9MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 23.7MB 45.7MB/s 
     |████████████████████████████████| 890kB 4.8MB/s 
     |████████████████████████████████| 14.7MB 49.6MB/s 
     |████████████████████████████████| 10.4MB 54.0MB/s 
     |████████████████████████████████| 184kB 5.0MB/s 
  Created wheel for pyshp: filename=pyshp-1.2.10-cp36-none-any.whl size=20467 sha256=46961f7afd0f3ea46356f083a8019872cdb2e2cf79f9039a46b0c956fffc03c1
  Stored in directory: /root/.cache/pip/wheels/ba/1a/67/6a12977f362c33a15edc753daf92c6f01879dbf4db76faf0dd
Successfully built pyshp


In [2]:
from __future__ import print_function

import numpy as np 
import chart_studio.plotly as py
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import datetime
pd.__version__


/home/abhay/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



'0.24.2'

In [3]:
covid_across_countries_df = pd.read_csv('COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')


In [4]:
global_timeseries = covid_across_countries_df.set_index(['Country/Region'])
global_timeseries.head(10)

,Province/State,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Thailand,NaN,15.0000,101.0000,2,3,5,7,8,8,14,...,43,47,48,50,50,50,53,59,70,75
Japan,NaN,36.0000,138.0000,2,1,2,2,4,4,7,...,331,360,420,461,502,511,581,639,639,701
Singapore,NaN,1.2833,103.8333,0,1,3,3,4,5,7,...,110,117,130,138,150,150,160,178,178,200
Nepal,NaN,28.1667,84.2500,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Malaysia,NaN,2.5000,112.5000,0,0,0,3,4,4,4,...,50,50,83,93,99,117,129,149,149,197
Canada,British Columbia,49.2827,-123.1207,0,0,0,0,0,0,1,...,12,13,21,21,27,32,32,39,46,64
Australia,New South Wales,-33.8688,151.2093,0,0,0,0,3,4,4,...,22,22,26,28,38,48,55,65,65,92
Australia,Victoria,-37.8136,144.9631,0,0,0,0,1,1,1,...,10,10,10,11,11,15,18,21,21,36
Australia,Queensland,-28.0167,153.4000,0,0,0,0,0,0,0,...,11,13,13,13,15,15,18,20,20,35


In [5]:
us_timeseries = global_timeseries.loc['US']
us_timeseries.head(5)

,Province/State,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
US,Washington,47.4009,-121.4905,0,0,0,0,0,0,0,...,0,0,0,0,0,0,267,366,442,568
US,New York,42.1657,-74.9481,0,0,0,0,0,0,0,...,0,0,0,0,0,0,173,220,328,421
US,California,36.1162,-119.6816,0,0,0,0,0,0,0,...,0,0,0,0,0,0,144,177,221,282
US,Massachusetts,42.2302,-71.5301,0,0,0,0,0,0,0,...,0,0,0,0,0,0,92,95,108,123
US,Diamond Princess,35.4437,139.6380,0,0,0,0,0,0,0,...,45,45,45,45,45,45,46,46,46,46


Now that we have the US data isloated, we can input this into a spatio-temporal map: Space represented as a map, and temporal as an animation.


In [6]:
#!pip install us
#import us

In [7]:
!wget http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?#
state_populations = pd.read_csv('nst-est2019-alldata.csv?', index_col='NAME')
state_populations.head(10)



--2020-03-18 12:15:49--  http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?
Resolving www2.census.gov (www2.census.gov)... 2405:200:1630:696::208c, 2405:200:1630:6a5::208c, 184.26.194.197
Connecting to www2.census.gov (www2.census.gov)|2405:200:1630:696::208c|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?&sec_ak_reference=18.27d82c31.1584513955.5e45998d [following]
--2020-03-18 12:15:57--  https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?&sec_ak_reference=18.27d82c31.1584513955.5e45998d
Connecting to www2.census.gov (www2.census.gov)|2405:200:1630:696::208c|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘nst-est2019-alldata.csv?’

nst-est2019-alldata     [   <=>    

,SUMLEV,REGION,DIVISION,STATE,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
NAME,,,,,,,,,,,,,,,,,,,,,
United States,10,0,0,0,308745538,308758105,309321666,311556874,313830990,315993715,...,0.000000,2.493773,2.682083,2.636187,2.921500,3.260435,3.252788,2.871957,2.153911,1.818059
Northeast Region,20,1,0,0,55317240,55318443,55380134,55604223,55775216,55901806,...,-5.254530,0.887909,-0.038355,-0.469783,-0.986097,-2.061965,-2.490484,-1.837048,-2.134447,-2.859713
Midwest Region,20,2,0,0,66927001,66929725,66974416,67157800,67336743,67560379,...,-2.365881,-0.963930,-0.973943,-0.006924,-0.762969,-1.388437,-1.241784,-0.557370,-0.922755,-1.111173
South Region,20,3,0,0,114555744,114563030,114866680,116006522,117241208,118364400,...,3.261349,5.130513,5.850458,5.292073,6.161501,7.277358,7.150074,6.198168,5.225519,5.203720
West Region,20,4,0,0,71945553,71946907,72100436,72788329,73477823,74167130,...,0.614245,2.723344,3.062896,3.162262,4.026429,4.987285,5.261078,4.021194,3.044951,2.312083
Alabama,40,3,6,1,4779736,4780125,4785437,4799069,4815588,4830081,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
Alaska,40,4,9,2,710231,710249,713910,722128,730443,737068,...,-12.929847,0.587728,1.416798,-0.955359,-11.460949,-7.997118,-3.897349,-10.992765,-13.859140,-12.031221
Arizona,40,4,8,4,6392017,6392288,6407172,6472643,6554978,6632764,...,12.609078,4.278167,6.899802,6.376679,9.168478,9.597577,11.964782,10.878879,12.962934,13.687161
Arkansas,40,3,7,5,2915918,2916031,2921964,2940667,2952164,2959400,...,0.834503,3.294766,0.827785,0.057853,-0.091449,1.075446,1.486269,2.009593,0.958896,0.923429


Get append data including population, hospitals, hospital beds.

In [9]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

good_states = state_populations.index.intersection(us_state_abbrev.keys())

us_state_pop = {}
for state_name in good_states:
    state_row = state_populations.loc[state_name]
    us_state_pop[state_name]=state_row["POPESTIMATE2019"]

print (us_state_pop)

{'Alabama': 4903185, 'Alaska': 731545, 'Arizona': 7278717, 'Arkansas': 3017804, 'California': 39512223, 'Colorado': 5758736, 'Connecticut': 3565287, 'Delaware': 973764, 'District of Columbia': 705749, 'Florida': 21477737, 'Georgia': 10617423, 'Hawaii': 1415872, 'Idaho': 1787065, 'Illinois': 12671821, 'Indiana': 6732219, 'Iowa': 3155070, 'Kansas': 2913314, 'Kentucky': 4467673, 'Louisiana': 4648794, 'Maine': 1344212, 'Maryland': 6045680, 'Massachusetts': 6892503, 'Michigan': 9986857, 'Minnesota': 5639632, 'Mississippi': 2976149, 'Missouri': 6137428, 'Montana': 1068778, 'Nebraska': 1934408, 'Nevada': 3080156, 'New Hampshire': 1359711, 'New Jersey': 8882190, 'New Mexico': 2096829, 'New York': 19453561, 'North Carolina': 10488084, 'North Dakota': 762062, 'Ohio': 11689100, 'Oklahoma': 3956971, 'Oregon': 4217737, 'Pennsylvania': 12801989, 'Rhode Island': 1059361, 'South Carolina': 5148714, 'South Dakota': 884659, 'Tennessee': 6829174, 'Texas': 28995881, 'Utah': 3205958, 'Vermont': 623989, 'Vi

In [10]:
hospital_beds_df = pd.read_csv('https://raw.githubusercontent.com/daveselinger/covid-19-hackathon/master/Hospitals.csv')

In [11]:
hospital_beds_df.replace(to_replace=-999, value=1, inplace=True)
print (hospital_beds_df.set_index('STATE').loc['AK'].head(5))
print (hospital_beds_df.groupby('TYPE').sum())

hospital_beds_df.index.value_counts()

hospital_bed_count_df = hospital_beds_df.where(hospital_beds_df['TYPE'] == "GENERAL ACUTE CARE").groupby('STATE')['BEDS'].sum()

                X          Y  OBJECTID       ID  \
STATE                                             
AK    -151.077827  60.493169       635   699669   
AK    -149.800293  61.182950       636   799508   
AK    -131.685788  55.353719       637   499901   
AK    -147.739393  64.831745       638  1299701   
AK    -165.405315  64.501918       639  2399762   

                                       NAME               ADDRESS       CITY  \
STATE                                                                          
AK       CENTRAL PENINSULA GENERAL HOSPITAL    250 HOSPITAL PLACE   SOLDOTNA   
AK      ALASKA NATIVE MEDICAL CENTER - ANMC  4315 DIPLOMACY DRIVE  ANCHORAGE   
AK     PEACEHEALTH KETCHIKAN MEDICAL CENTER   3100 TONGASS AVENUE  KETCHIKAN   
AK              FAIRBANKS MEMORIAL HOSPITAL    1650 COWLES STREET  FAIRBANKS   
AK           NORTON SOUND REGIONAL HOSPITAL      306 WEST 5TH AVE       NOME   

         ZIP           ZIP4       TELEPHONE  ...                  VAL_DATE  \
STA

Some key assumptions in computing these data:
- 40% of cases will need hospitalization
- 15% of cases will need an ICU
- ICU's are 15% of total beds in a hospital
- 90% of the cases are in the key metropolitan area of the state (which is where the outbreak will occur in a state most likely due to virality/spread -- this is NOT ACTUALLY the case if we had metro/county level data which would be much more accurate)
- 40% of the population (and importantly here hospital beds/icu beds) are in this metropolitan areas

In [12]:
def create_dfs(case_concentration=1.0, population_concentration=1.0):
  us_plotseries = []
  us_growthseries =[]
  startdate = datetime.date(2020,3,10)
  enddate = datetime.date(2020,3,14)
  days = (enddate - startdate).days
  for index, state_row in us_timeseries.iterrows():
    state_name = state_row["Province/State"]
    if state_name not in us_state_abbrev:
      continue
    state_abbrev = us_state_abbrev[state_name]
    last_count = 0
    growth_rate = 0.0
    for date in (startdate + datetime.timedelta(n) for n in range(days)):
      datestr = "" + str(date.month) + "/" + str(date.day) + "/" + str(date.year)[2:]
      count = state_row[datestr]
      row = [state_abbrev, str(date), count]
      if date.day == 9:
        print(row)
      us_plotseries.append(row)
      ## ROGER: Please compute over 2 days instead of 1
      if last_count > 0:
        growth_rate = 100.0 * count / last_count - 100
      last_count = count

    doubling_days = 10
    if growth_rate > 0:
      doubling_days = np.log(2)/np.log(1+ growth_rate/100.0)
    
    population = us_state_pop[state_name]
    percent_population = count * 10000.0 / population

    beds = hospital_bed_count_df.loc[state_abbrev]
    beds_in_metro = beds * population_concentration
    cases_needing_beds_in_metro = count * case_concentration * .4
    percent_beds = cases_needing_beds_in_metro * 100.0 / (beds_in_metro)

    icu_beds = beds * 0.15
    cases_needing_icu_in_metro = count * case_concentration * .15
    icu_beds_in_metro = icu_beds * population_concentration
    percent_icu = cases_needing_icu_in_metro * 100.0 / (icu_beds_in_metro)

# Now let's compute the number of days until the ICU is at 100%
    import math
    multiplier_to_fill_icu = 1.0 / (percent_icu/100)
    days_to_fill=100
    if growth_rate > 0:
      print ("For State " + state_abbrev + " need " + str(multiplier_to_fill_icu) + " growth to fill the ICU")
      print ("Growth rate" + str (growth_rate) + ";")
      days_to_fill = math.log10(multiplier_to_fill_icu)/math.log10(1+growth_rate/100.0)

    growth_row = [state_abbrev, str(date), count, growth_rate, doubling_days, population, percent_population, beds, percent_beds, icu_beds, percent_icu, days_to_fill]
  #  print (growth_row)
    us_growthseries.append(growth_row)

  us_plotseries_df = pd.DataFrame(us_plotseries, columns =['State', 'Day','Count'], dtype = int) 
  us_growthseries_df = pd.DataFrame(us_growthseries, columns =['State', 'Day','Count', 'Growth Rate', 'Days to Double', 'Population', 'Population Percent (1/100 of 1%)', 'Beds', 'Beds Percent', 'ICU', 'ICU Percent', 'Days To ICU Capacity'], dtype = int) 

  return us_plotseries_df, us_growthseries_df

us_plotseries_df, us_growthseries_df = create_dfs(case_concentration=0.9, population_concentration=0.4)
print (us_growthseries_df.head(10))


For State WA need 8.83568075117371 growth to fill the ICU
Growth rate28.50678733031674;
For State NY need 55.07310636051729 growth to fill the ICU
Growth rate28.353658536585357;
For State CA need 138.55949566587864 growth to fill the ICU
Growth rate27.601809954751133;
For State MA need 54.80036133694671 growth to fill the ICU
Growth rate13.888888888888886;
For State GA need 254.61375661375655 growth to fill the ICU
Growth rate35.48387096774192;
For State CO need 76.34467120181405 growth to fill the ICU
Growth rate8.888888888888886;
For State FL need 527.0133333333333 growth to fill the ICU
Growth rate42.85714285714286;
For State OR need 103.46666666666667 growth to fill the ICU
Growth rate25.0;
For State TX need 705.8501291989663 growth to fill the ICU
Growth rate59.25925925925927;
For State IL need 295.3913043478261 growth to fill the ICU
Growth rate43.75;
For State PA need 406.4932249322494 growth to fill the ICU
Growth rate86.36363636363637;
For State IA need 293.90849673202615 grow

/home/abhay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning:

divide by zero encountered in double_scalars



In [14]:
fig = px.choropleth(us_plotseries_df, title = "Cases", locations="State", locationmode="USA-states",scope="usa", color="Count", animation_frame="Day", range_color=[0,250])
fig.show()

In [15]:
fig2 = px.choropleth(us_growthseries_df, title="Growth Rate", locations="State", locationmode="USA-states",scope="usa", color="Growth Rate", range_color=[0,100])
fig2.show()

In [16]:
fig3 = px.choropleth(us_growthseries_df, title="Percent Infected (bps)", locations="State", locationmode="USA-states",scope="usa", color="Population Percent (1/100 of 1%)", range_color=[0,1])
fig3.show()

In [17]:
fig4 = px.choropleth(us_growthseries_df, title="Percent of Beds (%)", locations="State", locationmode="USA-states",scope="usa", color="Beds Percent", range_color=[0,50])
fig4.show()

In [18]:
fig5 = px.choropleth(us_growthseries_df, title="Percent of ICU Beds (%)", locations="State", locationmode="USA-states",scope="usa", color="ICU Percent", range_color=[0,50])
fig5.show()

In [19]:
fig6 = px.choropleth(us_growthseries_df, title="Days to fill ICU", color_continuous_scale='ylgnbu', locations="State", locationmode="USA-states",scope="usa", color="Days To ICU Capacity", range_color=[0,50])
fig6.show()

In [0]:
##ROGER

Next, let's compute a simulation to determine what the R0 we need for this state to NOT overwhelm the ICU.
This simulation will have to be done including  assumed length of stay in the ICU (14 days for now--can correct if we need to) (people in then people out).

